# Multi-head attention

**Descrição**  
Neste notebook, vamos implementar e explorar **multi-head attention** (atenção multi-cabeças). A ideia é sair do “resultado final” e abrir a caixa-preta do mecanismo de atenção, visualizando o que acontece em **cada cabeça (head)** separadamente.

**Objetivo**  
- Construir um pipeline mínimo: tokenização → índices → embeddings → multi-head attention.  
- Entender **conceitualmente** por que usar várias cabeças (heads) em vez de uma só.  
- Inspecionar, de forma interpretável, os principais componentes internos:
  - **matriz de atenção** de cada head (quem olha para quem)
  - **vetores de contexto** produzidos por cada head (o que cada head “constrói”)

**Funcionamento**  
1. **Tokenização e vocabulário**: a frase é separada por espaços (`split`) e mapeada para IDs (`vocab` / `inv_vocab`).  
2. **Embeddings**: cada token ID é convertido em um vetor denso com dimensão pequena para facilitar a visualização.  
3. **Projeções Q, K, V**: o embedding é projetado em **Queries (Q)**, **Keys (K)** e **Values (V)**.  
4. **Divisão em múltiplas cabeças**: Q/K/V são reorganizados para criar `num_heads` subconjuntos (“heads”), cada um com dimensão `head_dim`.  
5. **Atenção por head**: cada head calcula sua própria matriz de atenção e gera seus próprios vetores de contexto.  
6. **Concatenação e projeção final**: as saídas das heads são concatenadas e combinadas por uma camada linear (`out_proj`) para produzir a saída final do bloco de atenção.

![O gato sobe no tapete](../../imagens/cap03/04_gato_sobe_no_tapete.png)

In [1]:
import pandas as pd
import torch
import torch.nn as nn

# Para reprodutibilidade
torch.manual_seed(42)

## Frase de exemplo e vocabulário

In [2]:
# Frase de exemplo
sentence = "O gato sobe no tapete".split()

# Vocabulário
vocab = {word: idx for idx, word in enumerate(sentence)}
inv_vocab = {idx: word for word, idx in vocab.items()}

print("Vocabulário:")
for k, v in vocab.items():
    print(f"{k} -> {v}")

Vocabulário:
O -> 0
gato -> 1
sobe -> 2
no -> 3
tapete -> 4


## Conversão para índices (tokens)

In [3]:
# Convertendo palavras para índices
token_ids = torch.tensor([vocab[word] for word in sentence])

print("Tokens:")
for word, idx in zip(sentence, token_ids, strict=False):
    print(f"{word} -> {idx.item()}")

Tokens:
O -> 0
gato -> 1
sobe -> 2
no -> 3
tapete -> 4


## Camada de Embedding

In [4]:
vocab_size = len(vocab)
embedding_dim = 3  # dimensão pequena para fins didáticos

embedding = nn.Embedding(num_embeddings=vocab_size, embedding_dim=embedding_dim)

# Aplicando embedding
X = embedding(token_ids)

print("Shape dos embeddings:", X.shape)
X

Shape dos embeddings: torch.Size([5, 3])


tensor([[ 0.3367,  0.1288,  0.2345],
        [ 0.2303, -1.1229, -0.1863],
        [ 2.2082, -0.6380,  0.4617],
        [ 0.2674,  0.5349,  0.8094],
        [ 1.1103, -1.6898, -0.9890]], grad_fn=<EmbeddingBackward0>)

## O que é Multi-Head Attention?

Na **self-attention** “single-head”, você calcula uma única matriz de atenção:
- projeta o input para **Q** (queries), **K** (keys), **V** (values)
- computa similaridades `Q @ Kᵀ`
- aplica softmax (e máscara causal, se for o caso)
- mistura os valores `V` de acordo com os pesos de atenção

Na **multi-head attention**, em vez de fazer isso uma vez, você faz **em paralelo** em múltiplas “cabeças” (heads):
- cada head tem sua própria forma de projetar Q/K/V (na prática, uma projeção maior que é “fatiada”)
- cada head pode focar em padrões diferentes (ex.: concordância, dependências locais, estruturas sintáticas etc.)
- no final, você concatena as saídas das heads e aplica uma projeção final (`out_proj`)

Intuição curta:
- **single-head** = “um único tipo de olhar” sobre a sequência
- **multi-head** = “vários olhares em paralelo”, depois combinados

In [5]:
class MultiHeadAttention(nn.Module):
    """
    Implementa Multi-Head Causal Self-Attention (estilo Transformer) com máscara causal
    para impedir que cada token atenda tokens futuros.

    Parâmetros:
    ----------
    d_in : int
        Dimensão de entrada (features por token).
    d_out : int
        Dimensão total de saída (soma das dimensões de todas as heads).
        Deve ser divisível por num_heads.
    context_length : int
        Comprimento máximo de contexto (número máximo de tokens).
    dropout : float
        Probabilidade de dropout aplicada aos pesos de atenção.
    num_heads : int
        Número de cabeças de atenção.
    qkv_bias : bool, default = False
        Se True, adiciona bias nas camadas lineares de Q, K e V.

    Retorno:
    -------
    torch.Tensor
        Tensor de shape (batch_size, num_tokens, d_out) com os vetores de contexto.

    Exceções:
    --------
    Levanta ValueError se d_out não for divisível por num_heads.
    Levanta ValueError se x não tiver shape 3D (B, T, D).
    """

    def __init__(
        self,
        d_in: int,
        d_out: int,
        context_length: int,
        dropout: float,
        num_heads: int,
        qkv_bias: bool = False,
    ) -> None:
        super().__init__()

        if d_out % num_heads != 0:
            raise ValueError("d_out must be divisible by num_heads")

        self.d_out = int(d_out)
        self.num_heads = int(num_heads)
        self.head_dim = self.d_out // self.num_heads

        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)

        # Projeção final para combinar as heads
        self.out_proj = nn.Linear(d_out, d_out)

        self.dropout = nn.Dropout(dropout)

        # Máscara causal: 1s acima da diagonal principal (tokens futuros).
        self.register_buffer(
            "mask",
            torch.triu(torch.ones(context_length, context_length), diagonal=1),
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        if x.ndim != 3:
            raise ValueError(
                f"Esperado x com 3 dimensões (batch, tokens, d_in), mas veio shape={tuple(x.shape)}."
            )

        b, num_tokens, d_in = x.shape

        # Projeções: (b, T, d_out)
        keys = self.W_key(x)
        queries = self.W_query(x)
        values = self.W_value(x)

        # (b, T, d_out) -> (b, T, num_heads, head_dim)
        keys = keys.view(b, num_tokens, self.num_heads, self.head_dim)
        queries = queries.view(b, num_tokens, self.num_heads, self.head_dim)
        values = values.view(b, num_tokens, self.num_heads, self.head_dim)

        # (b, T, num_heads, head_dim) -> (b, num_heads, T, head_dim)
        keys = keys.transpose(1, 2)
        queries = queries.transpose(1, 2)
        values = values.transpose(1, 2)

        # Scores por head: (b, h, T, head_dim) @ (b, h, head_dim, T) -> (b, h, T, T)
        attn_scores = queries @ keys.transpose(2, 3)

        # Aplicar máscara causal (broadcast nas heads e batch)
        mask_bool = self.mask.bool()[:num_tokens, :num_tokens]
        attn_scores.masked_fill_(mask_bool, -torch.inf)

        # Softmax com escala por sqrt(d_k)
        attn_weights = torch.softmax(attn_scores / (keys.shape[-1] ** 0.5), dim=-1)
        attn_weights = self.dropout(attn_weights)

        # Contexto por head: (b, h, T, T) @ (b, h, T, head_dim) -> (b, h, T, head_dim)
        context = attn_weights @ values

        # (b, h, T, head_dim) -> (b, T, h, head_dim)
        context = context.transpose(1, 2)

        # Concatenar heads: (b, T, d_out)
        context = context.contiguous().view(b, num_tokens, self.d_out)

        # Projeção final
        context = self.out_proj(context)

        return context

## Rodando a MHA (com 2 heads)

Aqui a gente escolhe `d_out` divisível por `num_heads`. Como o embedding é `d_in=3`, vamos usar `d_out=4` e `num_heads=2 ⇒ head_dim=2`.

In [6]:
num_tokens = len(sentence)
context_length = num_tokens

mha = MultiHeadAttention(
    d_in=embedding_dim,
    d_out=4,  # precisa ser divisível por num_heads
    context_length=context_length,
    dropout=0.0,  # 0.0 pra ficar determinístico/didático
    num_heads=2,
    qkv_bias=False,
)

# A MHA espera (batch, tokens, d_in)
x_in = X.unsqueeze(0)  # (1, 5, 3)

y = mha(x_in)  # (1, 5, 4)
print("Input shape :", x_in.shape)
print("Output shape:", y.shape)
y

Input shape : torch.Size([1, 5, 3])
Output shape: torch.Size([1, 5, 4])


tensor([[[ 0.3455,  0.0400,  0.1735, -0.3224],
         [ 0.3484,  0.0101,  0.1389, -0.2539],
         [ 0.4340,  0.2990, -0.0576, -0.1211],
         [ 0.4144,  0.2152, -0.0041, -0.1653],
         [ 0.4142,  0.1889,  0.0125, -0.1536]]], grad_fn=<ViewBackward0>)

## O que existe “dentro” de cada head?

Para cada head `h`, existem:
- `Q_h`, `K_h`, `V_h` com shape `(tokens, head_dim)`
- `attn_scores_h = Q_h @ K_h^T` com shape `(tokens, tokens)`
- `attn_weights_h = softmax(attn_scores_h / sqrt(head_dim))` com shape `(tokens, tokens)`
- `head_context_h = attn_weights_h @ V_h` com shape `(tokens, head_dim)`

Depois:
- concatenamos `[head_context_0 || head_context_1]` → `(tokens, d_out)`
- aplicamos `out_proj` → `(tokens, d_out)`

⚠️ Importante:
Como estamos com pesos aleatórios (não treinamos nada), as atenções vão parecer “meio aleatórias”.
A graça aqui é **entender as peças e visualizar por head**.

### Função de inspeção

A ideia: usar os pesos da mha e reproduzir o forward passo-a-passo para capturar attn_weights e head_context de cada head.

In [7]:
@torch.no_grad()
def inspect_multihead_attention(
    mha: MultiHeadAttention, x: torch.Tensor, tokens: list[str]
):
    """
    Inspeciona Q/K/V, matriz de atenção e vetores de contexto por head,
    reproduzindo o forward com os pesos do módulo 'mha'.

    Parâmetros:
    ----------
    mha : MultiHeadAttention
        Módulo já instanciado (com pesos).
    x : torch.Tensor
        Entrada com shape (batch, tokens, d_in).
    tokens : list[str]
        Lista de tokens para rotular linhas/colunas.

    Retorno:
    -------
    dict com tensores intermediários.
    """
    b, num_tokens, _ = x.shape

    keys = mha.W_key(x)  # (b, tokens, d_out)
    queries = mha.W_query(x)  # (b, tokens, d_out)
    values = mha.W_value(x)  # (b, tokens, d_out)

    # split em heads
    keys = keys.view(b, num_tokens, mha.num_heads, mha.head_dim).transpose(
        1, 2
    )  # (b, heads, tokens, head_dim)
    queries = queries.view(b, num_tokens, mha.num_heads, mha.head_dim).transpose(1, 2)
    values = values.view(b, num_tokens, mha.num_heads, mha.head_dim).transpose(1, 2)

    attn_scores = queries @ keys.transpose(2, 3)  # (b, heads, tokens, tokens)

    mask_bool = mha.mask.bool()[:num_tokens, :num_tokens]
    attn_scores = attn_scores.masked_fill(mask_bool, float("-inf"))

    attn_weights = torch.softmax(
        attn_scores / (mha.head_dim**0.5), dim=-1
    )  # (b, heads, tokens, tokens)
    head_context = attn_weights @ values  # (b, heads, tokens, head_dim)

    # concat heads + out_proj (igual ao forward)
    concat_context = (
        head_context.transpose(1, 2).contiguous().view(b, num_tokens, mha.d_out)
    )
    out = mha.out_proj(concat_context)

    return {
        "queries": queries,
        "keys": keys,
        "values": values,
        "attn_scores": attn_scores,
        "attn_weights": attn_weights,
        "head_context": head_context,
        "concat_context": concat_context,
        "out": out,
    }


ins = inspect_multihead_attention(mha, x_in, sentence)
print("attn_weights shape:", ins["attn_weights"].shape)
print("head_context shape:", ins["head_context"].shape)

attn_weights shape: torch.Size([1, 2, 5, 5])
head_context shape: torch.Size([1, 2, 5, 2])


### Mostrar a matriz de atenção do Head 0 e Head 1

In [8]:
def show_head_attention_tables(ins_dict, tokens):
    attn = ins_dict["attn_weights"][0]  # (heads, tokens, tokens)
    num_heads = attn.shape[0]

    for h in range(num_heads):
        df = pd.DataFrame(
            attn[h].cpu().numpy(),
            index=[f"Q:{t}" for t in tokens],
            columns=[f"K:{t}" for t in tokens],
        )
        print(f"\n=== Head {h} | Matriz de atenção (attn_weights) ===")
        display(df.round(3))


show_head_attention_tables(ins, sentence)


=== Head 0 | Matriz de atenção (attn_weights) ===


,K:O,K:gato,K:sobe,K:no,K:tapete
Q:O,1.000,0.000,0.000,0.000,0.000
Q:gato,0.495,0.505,0.000,0.000,0.000
Q:sobe,0.285,0.266,0.449,0.000,0.000
Q:no,0.259,0.238,0.252,0.251,0.000
Q:tapete,0.177,0.193,0.249,0.183,0.198



=== Head 1 | Matriz de atenção (attn_weights) ===


,K:O,K:gato,K:sobe,K:no,K:tapete
Q:O,1.000,0.000,0.000,0.000,0.000
Q:gato,0.457,0.543,0.000,0.000,0.000
Q:sobe,0.346,0.418,0.236,0.000,0.000
Q:no,0.236,0.211,0.305,0.248,0.000
Q:tapete,0.199,0.286,0.091,0.166,0.258


### Mostrar os vetores de contexto produzidos por cada head

Interpretação:

* Cada head produz um vetor por token (aqui com head_dim=2).
* Esse vetor é uma “mistura” dos V (values) ponderada pelos attn_weights daquele head.

In [9]:
def show_head_context_vectors(ins_dict, tokens):
    ctx = ins_dict["head_context"][0]  # (heads, tokens, head_dim)
    num_heads = ctx.shape[0]

    for h in range(num_heads):
        df = pd.DataFrame(
            ctx[h].cpu().numpy(),
            index=[f"{t}" for t in tokens],
            columns=[f"dim{j}" for j in range(ctx.shape[-1])],
        )
        print(f"\n=== Head {h} | Vetores de contexto (head_context) ===")
        display(df.round(3))


show_head_context_vectors(ins, sentence)


=== Head 0 | Vetores de contexto (head_context) ===


,dim0,dim1
O,0.166,-0.226
gato,0.067,-0.143
sobe,0.317,-0.580
no,0.294,-0.455
tapete,0.207,-0.429



=== Head 1 | Vetores de contexto (head_context) ===


,dim0,dim1
O,0.076,-0.059
gato,0.160,-0.299
sobe,0.265,-0.402
no,0.271,-0.355
tapete,0.242,-0.386


### Mostrar concatenação das heads + saída final

In [10]:
concat_df = pd.DataFrame(
    ins["concat_context"][0].cpu().numpy(),
    index=sentence,
    columns=[f"concat_dim{j}" for j in range(ins["concat_context"].shape[-1])],
)
print("=== Concatenação [head0 || head1] (antes do out_proj) ===")
display(concat_df.round(3))

out_df = pd.DataFrame(
    ins["out"][0].cpu().numpy(),
    index=sentence,
    columns=[f"out_dim{j}" for j in range(ins["out"].shape[-1])],
)
print("\n=== Saída final (depois do out_proj) ===")
display(out_df.round(3))

=== Concatenação [head0 || head1] (antes do out_proj) ===


,concat_dim0,concat_dim1,concat_dim2,concat_dim3
O,0.166,-0.226,0.076,-0.059
gato,0.067,-0.143,0.160,-0.299
sobe,0.317,-0.580,0.265,-0.402
no,0.294,-0.455,0.271,-0.355
tapete,0.207,-0.429,0.242,-0.386



=== Saída final (depois do out_proj) ===


,out_dim0,out_dim1,out_dim2,out_dim3
O,0.346,0.040,0.173,-0.322
gato,0.348,0.010,0.139,-0.254
sobe,0.434,0.299,-0.058,-0.121
no,0.414,0.215,-0.004,-0.165
tapete,0.414,0.189,0.012,-0.154
